In [1]:
import boto3 
import sagemaker
from ibov.utils import load_config
from sagemaker.pytorch import PyTorch

### Loading Configs

In [2]:
# Load config dict
config = load_config()

In [3]:
role = config.get("sagemaker").get("role")
prefix = config.get("sagemaker").get("bucket_prefx")
data_dir = config.get("data").get("dir")
region = config.get("sagemaker").get("region")

### Upload Data

In [4]:
session = sagemaker.Session(boto_session=boto3.session.Session(region_name=region))
bucket = session.default_bucket()

In [5]:
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_config = session.upload_data(path="config.json", bucket=bucket, key_prefix=prefix)

### Train Model

In [13]:
estimator = PyTorch(entry_point="train.py", 
                    source_dir="ibov", 
                    py_version="py3",
                    role=role, 
                    framework_version='0.4.0',
                    instance_count=1, 
                    instance_type='ml.p2.xlarge')

In [ ]:
estimator.fit({'train': input_data, "config": input_config})

2021-01-14 02:20:19 Starting - Starting the training job...
2021-01-14 02:20:44 Starting - Launching requested ML instancesProfilerReport-1610590817: InProgress
.....

predictor = estimator.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge")